### [T&uuml;rme von Hanoi](https://de.wikipedia.org/wiki/T%C3%BCrme_von_Hanoi)
Ein Scheibenstapel soll verschoben werden. Dabei sind folgende Regeln einzuhalten:
- Es darf jeweils nur eine Scheibe verschoben werden.
- Auf eine Scheibe darf nur eine kleinere Scheibe gelegt werden.
- Es d&uuml;rfen max. 3 Stapel vorhanden sein.

Wir programmieren eine einfache, aber ausbauf&auml;hige Variante dieses Spiels.  
- Verschoben wird ein Stabel mit 4 Scheiben,
  repr&auml;sentiert durch die Zahlen 0, 1, 2 und 3. 
  Je gr&ouml;sser die Zahl,
  desto gr&ouml;sser die Scheibe.
- Eine absteigend sortierte Liste `[3, 2, 1, 0]` entspricht einem Stapel.
  Die letzte Zahl entspricht der obersten und kleinsten Scheibe.

### Der Kern des Spiels
Der Spielzustand wird in der Variable `stacks` gespeichert.
Diese enth&auml;lt eine Liste mit 3 Listen, den Stapeln.
Die Funktion `new_game()` startet ein neues Spiel, indem die Liste `stacks` modifiziert wird, so dass sie die 3 Stapel
`[3, 2, 1, 0]`, `[]` und `[]` enth&auml;lt.

Die Funktion `move_disk(src, dst)` verschiebt eine Scheibe von Stapel `src` (source) auf den Stapel `dst` (destination), falls regelkonform.

In [ ]:
ndisks = 4
stacks = []


def new_game():
    stacks[:] = [list(range(ndisks))[::-1], [], []]


def move_disk(src, dst):
    if not stacks[src] or (stacks[dst] and stacks[dst][-1] < stacks[src][-1]):
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)

In [ ]:
new_game()
stacks

In [ ]:
move_disk(0, 2)
stacks

In [ ]:
move_disk(0, 1)
stacks

In [ ]:
move_disk(2, 1)
stacks

### Spielbeginn und Scheibenverschiebungen als Events
Wir k&ouml;nnen bereits ein neues Spiel starten, Scheiben verschieben und dann
die neue Stabelkonfiguration betrachten, indem wir die Liste `stacks` ausgeben.
Wir m&ouml;chten jedoch, dass der Spielzustand automatisch nach einem Neustart des Spiels und nach jeder Stapelverschiebung ausgegeben wird.  

Diese Aufgabe &uuml;bertragen wir einem Eventhandler.
Der Eventhandler ist eine Funktion, die mit den Argumenten `event` und `data`
aufgerufen wird. `event` ist dabei der Name des Events und `data` sind die 
f&uuml;r die Bearbeitung des Events relevanten Daten.


Im vorliegenden Fall haben wir 2 Events, Neustart und Stapelverschiebung.
Wir behandeln beide gleich. F&uuml;rs erste geben wir jeweils eine textiche Darstellung der Stapelkonfiguration aus.

In [ ]:
ndisks = 4
stacks = []


def print_event(event, data):
    print(event, data)


event_handler = print_event


def new_game():
    stacks[:] = [list(range(ndisks))[::-1], [], []]
    event_handler('new game', stacks)


def move_disk(src, dst):
    if not stacks[src] or (stacks[dst] and stacks[dst][-1] < stacks[src][-1]):
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)
    event_handler('update stacks', stacks)

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)

In [ ]:
move_disk(0, 1)

In [ ]:
move_disk(2, 1)

### Textliche Darstellung der Stapel in einem Output-Widget

In [51]:
from ipywidgets import Output


layout = {'border': '1px solid black', 'height': '75px'}
out = Output(layout=layout)


@out.capture(clear_output=True)
def print_stacks(event, stacks):
    lines = []
    for h in range(ndisks):
        line = ''.join(' '*7 if len(stack) <= h else f'{'*' * (2*stack[h]+1):^7}'
                       for stack in stacks)
        lines.append(line)
    print('\n'.join(lines[::-1]))

In [52]:
out.clear_output()

In [53]:
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [54]:
event_handler = print_stacks

In [55]:
new_game()

In [56]:
move_disk(0, 2)

In [57]:
move_disk(0, 1)

In [58]:
move_disk(2, 1)

### Output-Widget zum debuggen von Fehlermeldungen der Callbacks

In [ ]:
err_out = Output(layout={'border': '1px solid black'})

### Steuerung mit Text-Widget

In [70]:
import warnings
from IPython.display import display
from ipywidgets import Text


prompt = 'Move ? '


@err_out.capture()  # zum debuggen
def on_press_enter(text):
    value = text.value.removeprefix(prompt)
    if value == 'n':
        new_game()
    if len(value) == 2 and value.isdigit():
        src, dst = (int(s)-1 for s in value)
        move_disk(src, dst)
    text.value = prompt


warnings.filterwarnings('ignore', category=DeprecationWarning)

text = Text(value=prompt)
text.on_submit(on_press_enter)

In [71]:
err_out.clear_output()

In [72]:
display(out, text, err_out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Text(value='Move ? ')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### Bildliche Darstellung der Stapelkonfiguration
Statt einer textlichen Darstellung der Stapelkonfiguration
soll der Eventhandler ein Bildliche Darstellung liefern.
Dazu weisen wir der Variable `event_handler` eine Funktion zu, welche
die Scheibenstapel unter Verwendung des Canvas-Objekts zeichnet.


In [95]:
from ipycanvas import Canvas


def draw_stack(stack, pos):
    h = disk_height
    for i, disk in enumerate(stack):
        canvas.fill_style = colors[disk]
        w = disk_widths[disk]
        canvas.fill_rect(pos - w/2, canvas.height-(i+1)*h, w, h)


@err_out.capture()  # zum debuggen
def draw_stacks(event, stacks):
    canvas.clear(1)
    for pos, stack in zip(stack_positions, stacks):
        draw_stack(stack, pos)


canvas_config = {
    'width': 300,
    'height': 100,
    'layout': {'border': '1px solid black'},
}

stack_positions = [50, 150, 250]
disk_widths = [30, 50, 70, 90]
disk_height = 10
colors = ['brown', 'teal', 'blue', 'purple']

canvas = Canvas(**canvas_config)

In [100]:
err_out.clear_output()

In [97]:
display(canvas, err_out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [98]:
event_handler = draw_stacks

In [ ]:
new_game()

In [89]:
move_disk(0, 2)

In [90]:
move_disk(0, 1)

In [91]:
move_disk(2, 1)

In [25]:
text

Text(value='Move ? ')

### Spiel mit der Tastatur steuern
Statt Spielstart und Scheibenverschiebung durch direkten Aufruf der Funktionen
`new_game()` und `move_disk(src, dst)` auszul&ouml;sen, m&ouml;chten wir
dies durch Dr&uuml;cken von `'n'`, bez. durch aufeinanderfolgendes Dr&uuml;cken 
zweier Stapelnummern erreichen. Die Zahlen 1, 2 und 3 stehen f&uuml;r die Stapel 0,1 und 2.

Dazu nutzen wir die F&auml;higkeit des Canvas-Widget auf Tastendr&uuml;cke zu h&ouml;ren.
Ist das Canvas-Widget aktive (passiert wenn man darauf klickt)
und wurde eine Funktion zum bearbeiten des Events `on_key_down` 
mit der Methode `on_key_down` registriert, wird beim Dr&uuml;cken einer Taste
diese Funktion aufgerufen. Das erste Argument ist der Tastennamen.

Zur Bearbeitung des Events `on_key_down` registrieren wir eine
Funktion `on_key_down`. Diese merkt sich in der globalen Variable `src` den Ausgangsstapel. Wird mit dem n&auml;chsten Tastendruck ein Zielstapel `dst` selektiert, so wird  `move_disk(src, dst)` aufgerufen.

In [26]:
# nur zum Debuggen
from canvas_callbacks import remove_callbacks

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [28]:
src = None


@err_out.capture()
def on_key_down(key, *flags):
    global src

    if key in '123' and src is None:
        src = int(key) - 1
        return
    elif key in '123':
        dst = int(key) - 1
        move_disk(src, dst)
    if key == 'n':
        new_game()
    src = None


remove_callbacks('on_key_down', canvas)
canvas.on_key_down(on_key_down)

In [29]:
err_out.clear_output()

In [30]:
display(canvas, err_out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### Maussteuerung hinzuf&uuml;gen

In [31]:
@err_out.capture()
def on_mouse_down(x, y):
    global src
    stack_width = canvas.width / 3
    src = int(x // stack_width)


@err_out.capture()
def on_mouse_up(x, y):
    global src

    if src is None:
        return

    stack_width = canvas.width / 3
    dst = int(x // stack_width)
    move_disk(src, dst)
    src = None


remove_callbacks('on_mouse_down', canvas)
remove_callbacks('on_mouse_up', canvas)
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_up(on_mouse_up)

In [48]:
err_out.clear_output()
out.clear_output()

In [50]:
display(canvas, out, text, err_out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Text(value='Move ? ')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [47]:
def show_both(event, data):
    draw_stacks(event, data)
    print_stacks(event, data)
    # print(event, data)  # debug info, show_both wird von on_... aufgerufen,
                        # output wir nach err_out umgeleitet


event_handler = show_both